In [ ]:
!pip install -qq transformers
!pip install -qq datasets
!pip install -qq accelerate
!pip install -qq bitsandbytes
!pip install -qq peft
!pip install -qq trl
!pip install -qq evaluate
!pip install -qq rouge_score
!pip install -qq jiwer
!pip install -qq wandb
!pip install -qq tensorboard
!pip install -qq gradio


In [ ]:
# !unzip /content/rlhf_dataset.zip

In [ ]:
import os


os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [ ]:
import warnings

warnings.filterwarnings("ignore")

In [ ]:
import random
import numpy as np

def set_seed(seed_val=42):
    random.seed(seed_val)
    np.random.seed(seed_val)
    torch.manual_seed(seed_val)
    torch.cuda.manual_seed_all(seed_val)

In [26]:
from huggingface_hub.hf_api import HfFolder
import os
# from google.colab import userdata
from kaggle_secrets import UserSecretsClient

# os.environ["HF_TOKEN"] = userdata.get("HUGGINGFACE_TOKEN")
os.environ["HF_TOKEN"] = UserSecretsClient().get_secret("HF_TOKEN")
HfFolder.save_token(os.environ["HF_TOKEN"])


In [27]:
import os
import wandb


# os.environ["WANDB_API_KEY"] = userdata.get("WANDB_API_KEY")
os.environ["WANDB_API_KEY"] = UserSecretsClient().get_secret("WANDB_API_KEY")

wandb.login(key=os.environ["WANDB_API_KEY"])

wandb_project = "RLHF_TinyLlama"
if len(wandb_project) > 0:
    os.environ["WANDB_PROJECT"] = wandb_project


wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: mohamed-ahmed. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
device

In [ ]:
import gc

def clear_gpu():
    torch.clear_autocast_cache()
    torch.cuda.ipc_collect()
    torch.cuda.empty_cache()
    gc.collect()

if device == "gpu":
    clear_gpu()

In [ ]:

import multiprocessing

def optimal_workers():
    num_cpus = multiprocessing.cpu_count()
    num_gpus = torch.cuda.device_count()
    optimal_value = min(num_cpus, num_gpus*4) if num_gpus else num_cpus - 1
    return optimal_value

num_cpu_workers = optimal_workers()
num_cpu_workers

In [ ]:
import torch

def set_seeds(seed: int=2):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

set_seeds(2)

## Step 1 - train policy model for human evaluation

In [ ]:
import pandas as pd


#df = pd.read_parquet("/content/rlhf_dataset/train_policy.parquet")
df = pd.read_parquet("/kaggle/input/rlhf-dataset/train_policy.parquet")
df.head()

In [ ]:
from datasets import Dataset

train_dataset = Dataset.from_pandas(df, split='train')
train_dataset

In [ ]:
import torch
from dataclasses import dataclass
from typing import Dict, List


@dataclass
class CustomDataCollator:
    def __init__(self, tokenizer, max_length, prompt_col, label_col):
        self.__tokenizer = tokenizer
        self.__max_length = max_length
        self.__prompt_col = prompt_col
        self.__label_col = label_col

    def __call__(self, samples: List[dict]):
        prompt_text = [str(s[self.__prompt_col]) for s in samples]
        label_text = [str(s[self.__label_col]) for s in samples]

        prompt_tokens = self.__tokenizer(prompt_text,
                                         return_tensors='pt',
                                         truncation=True,
                                         max_length=self.__max_length,
                                         padding="max_length")

        label_tokens = self.__tokenizer(label_text,
                                        return_tensors='pt',
                                        truncation=True,
                                        max_length=self.__max_length,
                                        padding="max_length")

        output_dict = dict()
        output_dict["input_ids"] = prompt_tokens['input_ids']
        output_dict["attention_mask"] = prompt_tokens['attention_mask']
        output_dict["labels"] = label_tokens['input_ids']
        return output_dict

In [ ]:
model_id = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
with_quantization_config = False if device == "cpu" else True
max_length = 512

In [ ]:
from transformers import AutoTokenizer


tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
custom_data_collator = CustomDataCollator(tokenizer=tokenizer,
                                          max_length=max_length,
                                          prompt_col="prompt",
                                          label_col="label")

In [ ]:
from transformers import AutoModelForCausalLM
from transformers import BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(load_in_4bit=True,
                                bnb_4bit_use_double_quant=True,
                                bnb_4bit_quant_type="nf4",
                                bnb_4bit_compute_dtype=torch.float16)

model = AutoModelForCausalLM.from_pretrained(model_id,
                                             quantization_config= bnb_config if with_quantization_config else None,
                                             device_map="auto",
                                             trust_remote_code=True,
                                             # torch_dtype=torch.float16
                                             )
model.config.pad_token_id = model.config.eos_token_id

In [ ]:
import bitsandbytes as bnb
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model, TaskType

def find_all_linear_names(model):
    cls = bnb.nn.Linear4bit
    lora_module_names = set()
    for name, module in model.named_modules():
        if isinstance(module, cls):
            names = name.split('.')
            lora_module_names.add(names[0] if len(names) == 1 else names[-1])
        if "lm_head" in lora_module_names: # Needed for 16bit
            lora_module_names.remove("lm_head")

    return list(lora_module_names)

def lora_peft_model(model, TaskType):
    target_modules = find_all_linear_names(model)

    peft_config = LoraConfig(r=128,  # dimension of the updated matrices
                             lora_alpha=32,  # parameter for scaling
                             target_modules=target_modules,  # this chooses on which layers QLoRA is applied
                             lora_dropout=0.05,  # dropout probability for layers
                             bias="none",
                             task_type=TaskType)


    ## model.enable_input_require_grads()
    model = prepare_model_for_kbit_training(model)
    model = get_peft_model(model, peft_config)
    model.print_trainable_parameters()
    return model

In [ ]:
model = lora_peft_model(model, TaskType.CAUSAL_LM)

In [ ]:
from transformers import TrainingArguments


output_dir = "tiny_llama_rlhf_policy_model"

training_arguments = TrainingArguments(output_dir=output_dir,
                                       learning_rate=1e-5,
                                       per_device_train_batch_size=16,
                                       fp16=False,
                                       gradient_accumulation_steps=1,
                                       save_strategy="steps",
                                       warmup_steps=50,
                                       logging_steps=20,
                                       max_steps=20, ###########################
                                       report_to=["tensorboard"],
                                       remove_unused_columns=False,
                                       save_total_limit=1,
                                       )

In [ ]:
from transformers import Trainer

trainer = Trainer(model=model,
                  args=training_arguments,
                  train_dataset=train_dataset,
                  data_collator=custom_data_collator)

In [ ]:
trainer.train()

In [ ]:
trainer.save_model("tiny_llama_rlhf_policy_model")
tokenizer.save_pretrained("tiny_llama_rlhf_policy_model")

In [ ]:
import torch
from transformers import AutoTokenizer
from transformers import AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel

fine_tunned_path = "tiny_llama_rlhf_policy_model"

model = AutoModelForCausalLM.from_pretrained(model_id,
                                             device_map="auto",
                                             torch_dtype=torch.float16)

merged_model= PeftModel.from_pretrained(model, fine_tunned_path)
merged_model = merged_model.merge_and_unload()

merged_model.save_pretrained("policy_merged_model", safe_serialiaztion=True)
tokenizer.save_pretrained("policy_merged_model", safe_serialiaztion=True)

### Restart The Kernel to save GPU Memory

## Step 2: train reward function

In [ ]:
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
device

In [ ]:
MODEL_PATH = "policy_merged_model"
with_quantization_config = False if device == "cpu" else True
max_length = 512

In [ ]:
import pandas as pd

# df = pd.read_parquet("/content/rlhf_dataset/train.parquet")
df = pd.read_parquet("/kaggle/input/rlhf-dataset/train.parquet")
df.head()

In [ ]:
from datasets import Dataset

raw_dataset = Dataset.from_pandas(df, split='train')
raw_dataset

In [ ]:
from transformers import AutoTokenizer


tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
import torch
from transformers import AutoModelForSequenceClassification
from transformers import BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(load_in_4bit=True,
                                bnb_4bit_use_double_quant=True,
                                bnb_4bit_quant_type="nf4",
                                bnb_4bit_compute_dtype=torch.float16)

model = AutoModelForSequenceClassification.from_pretrained(MODEL_PATH,
                                                          quantization_config= bnb_config if with_quantization_config else None,
                                                          device_map="auto",
                                                          trust_remote_code=True,
                                                          # torch_dtype=torch.float16
                                                          )
model.config.pad_token_id = model.config.eos_token_id

In [ ]:
import bitsandbytes as bnb
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model, TaskType

def find_all_linear_names(model):
    cls = bnb.nn.Linear4bit
    lora_module_names = set()
    for name, module in model.named_modules():
        if isinstance(module, cls):
            names = name.split('.')
            lora_module_names.add(names[0] if len(names) == 1 else names[-1])
        if "lm_head" in lora_module_names: # Needed for 16bit
            lora_module_names.remove("lm_head")

    return list(lora_module_names)

def lora_peft_model(model, task_type):
    target_modules = find_all_linear_names(model)

    peft_config = LoraConfig(r=128,  # dimension of the updated matrices
                             lora_alpha=32,  # parameter for scaling
                             target_modules=target_modules,  # this chooses on which layers QLoRA is applied
                             lora_dropout=0.05,  # dropout probability for layers
                             bias="none",
                             task_type=task_type)


    ## model.enable_input_require_grads()
    model = prepare_model_for_kbit_training(model)
    model = get_peft_model(model, peft_config)
    model.print_trainable_parameters()
    return model

In [ ]:
model = lora_peft_model(model, TaskType.SEQ_CLS)

In [ ]:
import torch
from dataclasses import dataclass
from typing import Dict, List


@dataclass
class CustomDataCollator:
    def __init__(self, tokenizer, max_length, prompt_col, chosen_col, rejected_col):
        self.__tokenizer = tokenizer
        self.__max_length = max_length
        self.__prompt_col = prompt_col
        self.__chosen_col = chosen_col
        self.__rejected_col = rejected_col

    def __call__(self, samples: List[dict]):
        chosen_text = [str(s[self.__prompt_col] + '\n' + s[self.__chosen_col]) for s in samples]
        rejected_text = [str(s[self.__prompt_col] + '\n' + s[self.__rejected_col]) for s in samples]

        chosen_tokens = self.__tokenizer.encode_plus(
            chosen_text,
            return_tensors='pt',
            padding="max_length",
            max_length=self.__max_length,
            truncation=True)

        rejected_tokens = self.__tokenizer.encode_plus(
            rejected_text,
            return_tensors='pt',
            padding="max_length",
            max_length=self.__max_length,
            truncation=True)

        output_dict = dict()
        output_dict["input_ids_chosen"] = chosen_tokens['input_ids']
        output_dict["attention_mask_chosen"] = chosen_tokens['attention_mask']

        output_dict["input_ids_rejected"] = rejected_tokens['input_ids']
        output_dict["attention_mask_rejected"] = rejected_tokens['attention_mask']

        return output_dict

In [ ]:
custom_data_collator = CustomDataCollator(tokenizer=tokenizer,
                                          max_length=max_length,
                                          prompt_col="prompt",
                                          chosen_col="chosen",
                                          rejected_col="rejected")

In [ ]:
from transformers import TrainingArguments
from trl import RewardConfig

output_dir = "tiny_llama_rlhf_reward_model"

training_args = RewardConfig(output_dir=output_dir,
                            learning_rate=1e-5,
                            per_device_train_batch_size=2,
                            per_device_eval_batch_size=1,
                            gradient_accumulation_steps=1,
                            fp16=False,
                            save_strategy="steps",
                            logging_steps=20,
                            max_steps=20, ######################################
                            report_to=["tensorboard"],
                            remove_unused_columns=False,
                            save_total_limit=1)

In [ ]:
from trl import RewardTrainer

trainer = RewardTrainer(model=model,
                        args=training_args,
                        tokenizer=tokenizer,
                        train_dataset=raw_dataset,
                        data_collator=custom_data_collator)

In [ ]:
trainer.train()

In [ ]:
trainer.save_model("tiny_llama_rlhf_reward_model")
tokenizer.save_pretrained("tiny_llama_rlhf_reward_model")

In [ ]:
import torch
from transformers import AutoTokenizer
from transformers import AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel

model_id = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
fine_tunned_path = "tiny_llama_rlhf_reward_model"

model = AutoModelForCausalLM.from_pretrained(model_id,
                                             device_map="auto",
                                             torch_dtype=torch.float16)

merged_model= PeftModel.from_pretrained(model, fine_tunned_path)
merged_model = merged_model.merge_and_unload()

merged_model.save_pretrained("reward_merged_model", safe_serialiaztion=True)
tokenizer.save_pretrained("reward_merged_model", safe_serialiaztion=True)

In [ ]:
def get_score(model, tokenizer, prompt, response):
    instructions = tokenizer.encode_plus(prompt,
                                         response,
                                         padding="max_length",
                                         max_length=max_length,
                                         return_tensors="pt",
                                         truncation=True).to(device)

    with torch.no_grad():
        outputs = model(**instructions)
    logits = outputs[0]

    return logits

In [ ]:
prompt = raw_dataset[2]["prompt"]
example_chosen_response = raw_dataset[2]["chosen"]
example_rejected_response = raw_dataset[2]["rejected"]

In [ ]:
prompt

In [ ]:
example_chosen_response

In [ ]:
example_rejected_response

In [ ]:
loss1 = get_score(model=model,
                  tokenizer=tokenizer,
                  prompt=prompt,
                  response=example_chosen_response)

loss2 = get_score(model=model,
                  tokenizer=tokenizer,
                  prompt=prompt,
                  response=example_rejected_response)

In [ ]:
from torch import nn

loss = -nn.functional.logsigmoid(loss1 - loss2).mean()
loss

In [ ]:
tokenizer.decode(torch.max(loss1, axis=-1).indices[0])

In [ ]:
tokenizer.decode(torch.max(loss2, axis=-1).indices[0])

### Restart The Kernel to save GPU Memory

## Step 3: RL PPO model

In [1]:
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [2]:
MODEL_PATH = "reward_merged_model"
max_length = 512

In [3]:
import pandas as pd
from datasets import Dataset

# df = pd.read_parquet("/content/rlhf_dataset/train.parquet")
df = pd.read_parquet("/kaggle/input/rlhf-dataset/train.parquet")
dataset = Dataset.from_pandas(df)
dataset

Dataset({
    features: ['prompt', 'chosen', 'rejected'],
    num_rows: 92534
})

In [4]:
dataset = dataset.select(range(100))

In [5]:
from trl import PPOConfig


config = PPOConfig(model_name=MODEL_PATH,
                   steps=100,
                   learning_rate=1e-5,
                   mini_batch_size=4,
                   batch_size=16,
                   remove_unused_columns=False)

/opt/conda/lib/python3.10/site-packages/trl/trainer/ppo_config.py:207: FutureWarning: `PPOConfig` is deprecated and will be removed in the future. Please use `PPOv2Config` with `PPOv2Trainer` instead.
  warnings.warn(


In [6]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(config.model_name)
tokenizer.pad_token = tokenizer.eos_token

In [7]:
def tokenize(sample):
    sample["input_ids"] = tokenizer.encode(sample["prompt"],
                                           return_tensors="pt",
                                           truncation=True,
                                           padding="max_length",
                                           max_length=max_length)[0]
    return sample

dataset = dataset.map(tokenize, batched=False)
dataset = dataset.map(lambda x: {"query": tokenizer.decode(x["input_ids"])}, batched=False)
dataset.set_format("pytorch")


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [8]:
def collator(data):
    return dict((key, [d[key] for d in data]) for key in data[0])

In [9]:
from trl import AutoModelForCausalLMWithValueHead
from transformers import BitsAndBytesConfig

with_quantization_config = False if device == "cpu" else True

bnb_config = BitsAndBytesConfig(load_in_4bit=True,
                                bnb_4bit_use_double_quant=True,
                                bnb_4bit_quant_type="nf4",
                                bnb_4bit_compute_dtype=torch.float16)


ppo_model = AutoModelForCausalLMWithValueHead.from_pretrained(MODEL_PATH,
                                                             quantization_config= bnb_config if with_quantization_config else None,
                                                             device_map="auto",
                                                             trust_remote_code=True,
                                                              # torch_dtype=torch.float16
                                                             )
ppo_model.config.pad_token_id = ppo_model.config.eos_token_id

In [10]:
from trl import create_reference_model

ref_model = create_reference_model(ppo_model)
print(ppo_model)

AutoModelForCausalLMWithValueHead(
  (pretrained_model): LlamaForCausalLM(
    (model): LlamaModel(
      (embed_tokens): Embedding(32000, 2048)
      (layers): ModuleList(
        (0-21): 22 x LlamaDecoderLayer(
          (self_attn): LlamaSdpaAttention(
            (q_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
            (k_proj): Linear4bit(in_features=2048, out_features=256, bias=False)
            (v_proj): Linear4bit(in_features=2048, out_features=256, bias=False)
            (o_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
            (rotary_emb): LlamaRotaryEmbedding()
          )
          (mlp): LlamaMLP(
            (gate_proj): Linear4bit(in_features=2048, out_features=5632, bias=False)
            (up_proj): Linear4bit(in_features=2048, out_features=5632, bias=False)
            (down_proj): Linear4bit(in_features=5632, out_features=2048, bias=False)
            (act_fn): SiLU()
          )
          (input_layernorm): LlamaRM

In [11]:
import bitsandbytes as bnb

optimizer = bnb.optim.Adam8bit(ppo_model.parameters(), lr=config.learning_rate)

In [12]:
from trl import PPOTrainer


ppo_trainer = PPOTrainer(config=config,
                           model=ppo_model,
                           ref_model=ref_model,
                           tokenizer=tokenizer,
                           dataset=dataset,
                           data_collator=collator,
                           optimizer=optimizer)

/opt/conda/lib/python3.10/site-packages/trl/trainer/ppo_trainer.py:193: FutureWarning: `PPOTrainer` is deprecated and will be removed in trl v0.12. Please use `PPOv2Trainer` instead.
  warnings.warn(


In [13]:
generation_kwargs = {
    "min_length": 5,
    "top_k": 0.0,
    "top_p": 1.0,
    "do_sample": True
}

In [14]:
reward_kwargs = {
    "top_k": None, # Return all scores.
    "function_to_apply": "none", # want the raw logits without softmax.
    "batch_size": 8,
}

In [15]:
from trl.core import LengthSampler


output_min_length = 128
output_max_length = max_length
output_length_sampler = LengthSampler(output_min_length, output_max_length)

In [16]:
from transformers import pipeline


pipe = pipeline("text-classification",
                model=MODEL_PATH, 
                device=device,
               )

pipe.tokenizer.pad_token = pipe.tokenizer.eos_token
pipe.model.config.pad_token_id = pipe.model.config.eos_token_id

Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at reward_merged_model and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [17]:
from trl.core import LengthSampler


output_length_sampler = LengthSampler(124, max_length)

The operation is running if you see the following metrics appearing:

* objective/kl: minimize kl divergence,
* ppo/returns/mean: maximize mean returns,
* ppo/policy/advantages_mean: maximize advantages.

In [18]:
from tqdm import tqdm

max_ppo_steps = 10
label_idx = 0


for step, batch in tqdm(enumerate(ppo_trainer.dataloader)):
    # Break when you reach max_steps.
    if step >= max_ppo_steps:
        break   

    prompt_tensors = batch["input_ids"]

    response_tensors = []

    for prompt_tensor in prompt_tensors:
        max_new_tokens = output_length_sampler()        
            
        generation_kwargs["max_new_tokens"] = max_new_tokens
        response = ppo_trainer.generate(prompt_tensor, **generation_kwargs)
        
        response_tensors.append(response.squeeze()[-max_new_tokens:])
        
    # This needs to be called "response".
    batch["response"] = [tokenizer.decode(r.squeeze()) for r in response_tensors]

    # Compute reward outputs.
    query_response_pairs = [q + r for q, r in zip(batch["query"], batch["response"])]    
    rewards = pipe(query_response_pairs, **reward_kwargs)

    reward_tensors = [torch.tensor(reward[label_idx]["score"]) for reward in rewards]    

    # Run PPO step.
    stats = ppo_trainer.step(prompt_tensors, response_tensors, reward_tensors)
    ppo_trainer.log_stats(stats, batch, reward_tensors)
    
    print(f'objective/kl: {stats["objective/kl"]}')
    print(f'ppo/returns/mean: {stats["ppo/returns/mean"]}')
    print(f'ppo/policy/advantages_mean: {stats["ppo/policy/advantages_mean"]}')
    print('-'.join('' for x in range(100)))

0it [00:00, ?it/s]Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)
1it [03:45, 225.53s/it]

objective/kl: 0.0
ppo/returns/mean: 0.1757490634918213
ppo/policy/advantages_mean: 0.001481133047491312
---------------------------------------------------------------------------------------------------


/opt/conda/lib/python3.10/site-packages/trl/trainer/ppo_trainer.py:1313: UserWarning: KL divergence is starting to become negative: -12.62 - this might be a precursor for failed training. sometimes this happens because the generation kwargs are not correctly set. Please make sure that the generation kwargs are set correctly, or review your training hyperparameters.
  warnings.warn(
2it [06:33, 191.66s/it]

objective/kl: -12.618038177490234
ppo/returns/mean: 0.5381687879562378
ppo/policy/advantages_mean: -0.0008931616321206093
---------------------------------------------------------------------------------------------------


/opt/conda/lib/python3.10/site-packages/trl/trainer/ppo_trainer.py:1313: UserWarning: KL divergence is starting to become negative: -3.14 - this might be a precursor for failed training. sometimes this happens because the generation kwargs are not correctly set. Please make sure that the generation kwargs are set correctly, or review your training hyperparameters.
  warnings.warn(
3it [10:02, 199.68s/it]

...

objective/kl: 0.5264463424682617
ppo/returns/mean: 0.06373009085655212
ppo/policy/advantages_mean: 0.0011384936515241861
---------------------------------------------------------------------------------------------------


5it [18:11, 225.62s/it]

objective/kl: 3.87739896774292
ppo/returns/mean: 0.2750890552997589
ppo/policy/advantages_mean: -0.009287512861192226
---------------------------------------------------------------------------------------------------


6it [21:53, 218.93s/it]

objective/kl: -0.17352545261383057
ppo/returns/mean: 0.21748584508895874
ppo/policy/advantages_mean: 0.012525477446615696
---------------------------------------------------------------------------------------------------


In [19]:
batch_size = 16
compare_results = {}

df_batch = dataset[0:batch_size]

compare_results["query"] = df_batch["query"]
prompt_tensors = df_batch["input_ids"]

response_tensors_ref = []
response_tensors = []

# Get response from ppo and base model.
for i in tqdm(range(batch_size)):
    gen_len = output_length_sampler()
    generation_kwargs["max_new_tokens"] = gen_len
    
    response = ref_model.generate(
        input_ids=torch.as_tensor(prompt_tensors[i]).unsqueeze(dim=0).to(device), 
        **generation_kwargs
    ).squeeze()[-gen_len:]
    response_tensors_ref.append(response)

    response = ppo_model.generate(
        input_ids=torch.as_tensor(prompt_tensors[i]).unsqueeze(dim=0).to(device), 
        **generation_kwargs
    ).squeeze()[-gen_len:]
    response_tensors.append(response)

# Decode responses.
compare_results["response_before"] = [tokenizer.decode(response_tensors_ref[i]) for i in range(batch_size)]
compare_results["response_after"] = [tokenizer.decode(response_tensors[i]) for i in range(batch_size)]

# Sentiment analysis of query/response pairs before/after.
texts_before = [d + s for d, s in zip(compare_results["query"], compare_results["response_before"])]
rewards_before = pipe(texts_before, **reward_kwargs)
compare_results["reward_before"] = [reward[label_idx]["score"] for reward in rewards_before]

texts_after = [d + s for d, s in zip(compare_results["query"], compare_results["response_after"])]
rewards_after = pipe(texts_after, **reward_kwargs)
compare_results["reward_after"] = [reward[label_idx]["score"] for reward in rewards_after]

100%|██████████| 16/16 [03:21<00:00, 12.56s/it]


In [20]:
pd.set_option('display.max_colwidth', 500)
df_compare_results = pd.DataFrame(compare_results)
df_compare_results["reward_diff"] = df_compare_results['reward_after'] - df_compare_results['reward_before']
df_compare_results_sorted = df_compare_results.sort_values(by=['reward_diff'], ascending=False).reset_index(drop=True)
df_compare_results_sorted

,query,response_before,response_after,reward_before,reward_after,reward_diff
0,"<s> SUBREDDIT: r/AskReddit\nTITLE: Are people really as excitable as they seem in game shows?\nPOST: I keep the TV on in the background and I forgot to change the channel. Let's Make a Deal comes on, and I swear to $Deity, these people are having near orgasms for *everything*. \r\n\r\nWheel spinning to select a contestant? Most exciting thing ever!\r\n\r\nGet to actually *be* the next contestant? Undulate wildly as if having a seizure.\r\n\r\nPick a case? Jump up and down like it contains an...","Javascript: ether had to use a website health. It helps users to ask about whatever is offered come down under a positive screen sausage masterpiece. Users win not to tell reporters they want again. However, I couldn't forget to clear if it shot or popped out as a kind of useful storys which the app could be overturned. Can't be enlarged for amino usefulta news another story's much more beautifulasing tan of wise offerings a phaunas soles' ta see them provide of useful practical deductions' ...","= --> since the bits... forni os' allow the tests.. Cher lisp distribution randias at nospe classes must avail... In and/or osf: zo stroll v : o / os:s r:... .. In ns/os: o r:e/ ../ /os: o/os/ bea A / os: o/os/ otive: o: r/nz: ... ... av: ora:/. play dissten s r/nada: ....... .... In nfor: o:r/os: ... .... ...... ......... ............... .... Save&rsquo; ou: o/os/ beau: / os/ aber/ ... ... .... Do not... In_ o"" :er"" ... ... ... ... ... In Voor and/or osf: and...",2.295279,2.295279,0.000000e+00
1,"<s> SUBREDDIT: r/AskReddit\nTITLE: Are people really as excitable as they seem in game shows?\nPOST: I keep the TV on in the background and I forgot to change the channel. Let's Make a Deal comes on, and I swear to $Deity, these people are having near orgasms for *everything*. \r\n\r\nWheel spinning to select a contestant? Most exciting thing ever!\r\n\r\nGet to actually *be* the next contestant? Undulate wildly as if having a seizure.\r\n\r\nPick a case? Jump up and down like it contains an...","case? Jump up and down like it contains an actual genie.\r\n\r\nSomeone pick your case to open? That person is a closer friend to you than your entire family, scream encouragement at them!\r\n\r\nI just don't get it. They clearly tart it up for syndication, but I keep thinking I'm missing something here. Surely, people don't get this keyed-up over such trivialities.</s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s>...","Bayer, Philip Stanley.\nWeber, Mark Van (Host, Jim').\nOur company offidesign' across bayernallow is an investment that we design affectsusestuding' an investorment. Outside filtering, as-- foundvestor--eval--ara-- easlish•e and ividensur.\nOfussign' a------------eenvoldssion,' a--. Ouiasing' a future.' invself' ?. With evowaming' venpend' an differential',' over 'vidden' a any, haspend' neths--oopservlet--ud, as--very--apple eit--a------a.O saving' an task.' It'--- a non tank.' e ap--a))))-...",2.295279,2.295279,0.000000e+00
2,"<s> SUBREDDIT: r/AskReddit\nTITLE: Are people really as excitable as they seem in game shows?\nPOST: I keep the TV on in the background and I forgot to change the channel. Let's Make a Deal comes on, and I swear to $Deity, these people are having near orgasms for *everything*. \r\n\r\nWheel spinning to select a contestant? Most exciting thing ever!\r\n\r\nGet to actually *be* the next contestant? Undulate wildly as if having a seizure.\r\n\r\nPick a case? Jump up and down like it contains an...","in game shows?\nPOST: I keep the TV on in the background and I forgot to change the channel. Let's Make a Deal comes on, and I swear to $Deity, these people are having near orgasms for *everything*. \r\n\r\nWheel spinning to select a contestant? Most exciting thing ever!\r\n\r\nGet to actually *be* the next contestant? Undulate wildly as if having a seizure.\r\n\r\nPi

In [21]:
ppo_model.save_pretrained("RLHF Model")
tokenizer.save_pretrained("RLHF Model")

('RLHF Model/tokenizer_config.json',
 'RLHF Model/special_tokens_map.json',
 'RLHF Model/tokenizer.model',
 'RLHF Model/added_tokens.json',
 'RLHF Model/tokenizer.json')

In [24]:
ppo_model.push_to_hub("Tinny-Llama_RLHF_PPO")
tokenizer.push_to_hub("Tinny-Llama_RLHF_PPO")

model.safetensors:   0%|          | 0.00/762M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/MohamedAhmedAE/Tinny-Llama_RLHF_PPO/commit/49de69e156ba1736c43cb8517607c01d7b110f38', commit_message='Upload tokenizer', commit_description='', oid='49de69e156ba1736c43cb8517607c01d7b110f38', pr_url=None, repo_url=RepoUrl('https://huggingface.co/MohamedAhmedAE/Tinny-Llama_RLHF_PPO', endpoint='https://huggingface.co', repo_type='model', repo_id='MohamedAhmedAE/Tinny-Llama_RLHF_PPO'), pr_revision=None, pr_num=None)